In [14]:
#manipulate dataframes in python
import pandas as pd
import sys

#make API calls with python
import requests

# sys.path.insert(0, '/path/to/application/app/folder') used to import local python file, in this case local keys
sys.path.insert(0, '../src/')
import localvars

#allows us to store results of API call cleanly
import json

# Load the first sheet of the JSON file into a data frame
df = pd.read_json('../src/data/data.json', orient='index')

# View the first ten rows
print(df.head(10))

# List of Years
strYears =["2010","2013", "2014", "2015", "2016", "2017"]

intState = 47
intCounty = 125

for index, row in df.iterrows():
    tablename= row['table']
    #print("value: " + tablesdf2.loc[tablename,'file'])
    # Using for loop 
    metric = pd.DataFrame() #creates a new dataframe that's empty
    for i in strYears: 

        #construct the API call we will use
        if tablename[:1]=="B":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5?get=%s&for=tract:*&in=state:%i%%20county:%i&key=%s" 
        elif tablename[:1]=="D":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5/profile?get=%s&for=tract:*&in=state:%i%%20county:%i&key=%s" 

        calledAPI = (baseAPI % (i, tablename, intState, intCounty, localvars.apiKey))
        
        print(calledAPI)

        #call the API and collect the response
        response = requests.get(calledAPI)

        #load the response into a JSON, ignoring the first element which is just field labels
        try:
            formattedResponse = json.loads(response.text)[1:]

            #flip the order of the response from [population, zipcode] -> [zipcode, population]
            formattedResponse = [item[::-1] for item in formattedResponse]

            #store the response in a dataframe
            strColYear = "y_" + i

            metrictemp = pd.DataFrame(columns=['id', '2', '3', strColYear], data=formattedResponse)
            del metrictemp['2']
            del metrictemp['3']
            if i == "2010":
                metric=metrictemp
            else: 
                metric = metric.merge(metrictemp, on='id', how='outer')
        except:
            print("Unexpected error:", sys.exc_info()[0])
            raise

    #save that dataframe to a CSV spreadsheet
    strFileName=str(row['metric'])
    strFile = "../src/data/ProcessedData/r" + strFileName + ".csv"
    strJson ="../src/data/json/m" + strFileName[1:]+ ".json"
    metric.to_csv(strFile, index=False)
    print("Output: " + strFile)

   category   label  metric                 subtitle        table  \
m1   Totals   Total       1        Population Counts  B01001_001E   
m2   Totals  Totals       2    Total Male Population  B01001_002E   
m3   Totals  Totals       3  Total Female Population  B01001_026E   

               title type  
m1  Total Population  sum  
m2       Total Males  sum  
m3     Total Females  sum  
https://api.census.gov/data/2010/acs/acs5?get=B01001_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
https://api.census.gov/data/2013/acs/acs5?get=B01001_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
https://api.census.gov/data/2014/acs/acs5?get=B01001_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
https://api.census.gov/data/2015/acs/acs5?get=B01001_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
https://api.census.gov/data/2016/acs/acs5?get=B0100